# Homework set 2

Please **submit this Jupyter notebook through Canvas** no later than **Mon Nov. 14, 9:00**. **Submit the notebook file with your answers (as .ipynb file) and a pdf printout. The pdf version can be used by the teachers to provide feedback. A pdf version can be made using the save and export option in the Jupyter Lab file menu.**

Homework is in **groups of two**, and you are expected to hand in original work. Work that is copied from another group will not be accepted.

# Exercise 0
Write down the names + student ID of the people in your group.

Jade Dubbeld, 11692065

Maickel Hartlief, 14015277

Run the following cell to import some packages, add additional packages yourself when needed.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from math import isclose

# Exercise 1

## (a) 1 point
Let $A$ be the matrix $\begin{bmatrix} 1 & -1 & \alpha \\ 2 & 2 & 1 \\ 0 & \alpha & -3/2 \end{bmatrix}$. For which values of $\alpha$ is $A$ singular?

A n x n matrix $A$ is nonsingular if either of the following hold:
\
1. $A$ has an inverse
2. $det(A) ≠ 0$
3. $rank(A) = n$
4. For all values of $z$ in range $R^n$ (except for $z ≠ 0$), we have $Mz ≠ 0$
\
\
Therefore, to find the values of $\alpha$ for which matrix $A$ is singular, we can solve $det(A) = 0$.
\
\
Given n x n matrix $A$ as $\begin{bmatrix} 1 & -1 & \alpha \\ 2 & 2 & 1 \\ 0 & \alpha & -\frac{3}{2} \end{bmatrix}$, we can derive the determinant of $A$. The definition of the determinant is $det(A) = a_{11} a_{22} a_{33} + a_{12} a_{23} a_{31} + a_{13} a_{21} a_{32} - a_{13} a_{22} a_{31} - a_{12} a_{21} a_{33} - a_{11} a_{23} a_{32}$ given a general n x n matrix $\begin{bmatrix} a_{11} & a_{12} & a_{13} \\ a_{21} & a_{22} & a_{23} \\ a_{31} & a_{32} & a_{33} \end{bmatrix}$
\
\
For this case, we fill in the values of provided matrix $A$ in the definition of the determinant. Then, $det(A) = (1*2*-\frac{3}{2})+(-1*1*0)+(\alpha*2*\alpha)-(\alpha*2*0)-(-1*2*-\frac{3}{2})-(1*1*\alpha) = -3 + 0 + 2 \alpha^2 - 0 - 3 - \alpha = 2 \alpha^2 - \alpha - 6$
\
\
To find the values of $\alpha$ for which matrix $A$ is singular, we solve $det(A) = 0$.
\
$2 \alpha^2 - \alpha - 6 = 0$
\
$D = b^2 - 4 a c = (-1)^2 - 4*2*-6 = 49$
\
$\alpha = \frac{1+\sqrt{49}}{2*2} = 4 \lor \alpha = \frac{1-\sqrt{49}}{2*2}=-1\frac{1}{2}$
\
\
Thus, matrix $A$ is singular when $\alpha$ is either $2$ or $-1\frac{1}{2}$.

## (b) 1 point
For the largest value of $\alpha$ you found above, find a nonzero vector $b$ such that $Ax = b$ has infinitely many solutions. Explain your answer.

The expression $Ax = b$ has infinitely many solutions if, and only if, $A$ is singular and $b \in span(A)$. If this does not apply, $Ax = b$ has no solutions. For this exercise, the largest value of $\alpha$ found above is $\alpha = 2$. Then, matrix $A = \begin{bmatrix} 1 & -1 & 2 \\ 2 & 2 & 1 \\ 0 & 2 & -\frac{3}{2} \end{bmatrix}$.
\
To see how many solutions there are to the expression $Ax = b$ in this case, we choose an arbitrary vector $x$. Say, we choose vector $x = \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix}$.
\
Then, solve $Ax = b$ using matrix multiplication.
\
$\begin{bmatrix} 1 & -1 & 2 \\ 2 & 2 & 1 \\ 0 & 2 & -\frac{3}{2} \end{bmatrix} \begin{bmatrix} 1 \\ 2 \\ 3 \end{bmatrix} = \begin{bmatrix} 1*1 + -1*2 + 2*3 \\ 2*1 + 2*2 + 1*3 \\ 0*1 + 2*2 + -\frac{3}{2}*3 \end{bmatrix} = \begin{bmatrix} 5 \\ 9 \\ -\frac{1}{2} \end{bmatrix}$
\
\
We found a solution for $b$ with an arbitrary vector $x$. Thus, the $Ax = b$ has infinitely many solutions.

# Exercise 2

For solving linear systems such as $Ax = b$, it is unnecessary (and often unstable) to compute the inverse $A^{-1}$. Nonetheless, there can be situations where it is useful to compute $A^{-1}$ explicitly. One way to do so is by using the LU-decomposition of $A$.

## (a) 2 points
Write an algorithm to compute $A^{-1}$ for a non-singular matrix $A$ using its LU-decomposition. You can use `scipy.linalg.lu` (which returns an LU-decomposition with _partial pivoting_, i.e., with a permutation matrix $P$) and the other `scipy.linalg.lu_*` functions, but not `scipy.linalg.inv` (or other methods for computing matrix inverses directly).

(Make sure to import the necessary functions/packages.)

In [2]:
def forward_substitution(L, b):
    # initialize output
    y = np.empty(b.shape)
    
    # calculate first row.
    y[0] = b[0] / L[0, 0]
    
    # calculating the other rows
    for i in range(1, len(L)):
        y[i] = (b[i] - L[i,:i] @ y[:i]) / L[i,i]  
    
    return y


def back_substitution(U, y):    
    # initializing output
    x = np.zeros(y.shape)
    
    # calculate last row
    x[-1] = y[-1] / U[-1, -1]
    
    # calculte the other rows (in reverse order)
    for i in range(len(U) - 2, -1, -1):
        x[i] = (y[i] - U[i, i:] @ x[i:]) / U[i, i]

    return x


def invert_PLU(A):
    '''invert the matrix A'''
    n = len(A)
    P, L, U = linalg.lu(A)
    
    I = np.identity(n)
    A_inv = np.empty((n, n))

    for i in range(n):
        y = forward_substitution(L, I[i, :] @ P)
        A_inv[:, i] = back_substitution(U, y)
        
    return A_inv

A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
A_inv = linalg.inv(A)

assert np.isclose(invert_PLU(A), A_inv).all()

## (b) 1 point
What is the computational complexity of your algorithm, given that the input matrix has size $n \times n$?
Give a short calculation/explanation for your answer.

First, we do LU decomposition once. This algorithm has a time-complexity of $\mathcal{O}(n^3)$.


Then we do forward and backward substitutions $n$ times.

Both forward and backward substitution have a loop of size $n$, in which there is a dot-product that also has complexity $n$. This leads to:

$n(n*n + n*n) = n(2n^2) = 2n^3 \sim \mathcal{O}(n^3)$

Since we do LU decomposition once and then the above process once as well, we end up with:
$\mathcal{O}(n^3)$


# Exercise 3

## (a) (2 points) 
What happens when Gaussian elimination with partial pivoting is used on a matrix of the following form?
$$
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  1 \\
    -1 &  1 &  0 &  0 &  1 \\
    -1 & -1 &  1 &  0 &  1 \\
    -1 & -1 & -1 &  1 &  1 \\
    -1 & -1 & -1 & -1 &  1 
  \end{bmatrix}
$$
Do the entries of the transformed matrix grow? What happens if complete pivoting is used instead? (Note that part (a) does not require a computer.)


Given matrix $A$ =
$
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  1 \\
    -1 &  1 &  0 &  0 &  1 \\
    -1 & -1 &  1 &  0 &  1 \\
    -1 & -1 & -1 &  1 &  1 \\
    -1 & -1 & -1 & -1 &  1 
  \end{bmatrix}
$
we can obtain the upper and lower triangular matrix by applying Gaussian elimination (also known as LU decomposition) with partial pivoting. In partial pivoting, it is only allowed to rearrange rows in a matrix, not the columns, whereas in complete pivoting, it is allowed to rearrange both the rows and the columns.

#### Gaussian elimination with partial pivoting
$
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  1 \\
    -1 &  1 &  0 &  0 &  1 \\
    -1 & -1 &  1 &  0 &  1 \\
    -1 & -1 & -1 &  1 &  1 \\
    -1 & -1 & -1 & -1 &  1 
  \end{bmatrix}
$
&rarr;
$
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  1 \\
     0 &  1 &  0 &  0 &  2 \\
     0 & -1 &  1 &  0 &  2 \\
     0 & -1 & -1 &  1 &  2 \\
     0 & -1 & -1 & -1 &  2 
  \end{bmatrix}
$
&rarr;
$
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  1 \\
     0 &  1 &  0 &  0 &  2 \\
     0 &  0 &  1 &  0 &  2 \\
     0 &  0 & -1 &  1 &  4 \\
     0 &  0 & -1 & -1 &  4 
  \end{bmatrix}
$
&rarr;
$
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  1 \\
     0 &  1 &  0 &  0 &  2 \\
     0 &  0 &  1 &  0 &  2 \\
     0 &  0 &  0 &  1 &  8 \\
     0 &  0 &  0 & -1 &  8 
  \end{bmatrix}
$
&rarr;
$
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  1 \\
     0 &  1 &  0 &  0 &  2 \\
     0 &  0 &  1 &  0 &  2 \\
     0 &  0 &  0 &  1 &  8 \\
     0 &  0 &  0 &  0 &  16 
  \end{bmatrix}
$
&rarr;
$
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  1 \\
     0 &  1 &  0 &  0 &  2 \\
     0 &  0 &  1 &  0 &  2 \\
     0 &  0 &  0 &  1 &  8 \\
     0 &  0 &  0 &  0 &  1 
  \end{bmatrix}
$
$= U$
\
\
Summing all transformations, we can retrieve the lower triangular matrix.
\
$
L =
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  0 \\
    -1 &  1 &  0 &  0 &  0 \\
    -1 & -1 &  1 &  0 &  0 \\
    -1 & -1 & -1 &  1 &  0 \\
    -1 & -1 & -1 & -1 &  \frac{1}{16} 
  \end{bmatrix}    
$
\
\
Specifically with this matrix $A$, there are no rows rearranged. Hence, no permutations were required and no permutation matrix can be derived. The entries of the transformed matrix did grow. The last column began to grow by $2n$ for each operation on the matrix.

#### Gaussian elimination with complete pivoting
$
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  1 \\
    -1 &  1 &  0 &  0 &  1 \\
    -1 & -1 &  1 &  0 &  1 \\
    -1 & -1 & -1 &  1 &  1 \\
    -1 & -1 & -1 & -1 &  1 
  \end{bmatrix}
$
&rarr;
$
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  1 \\
     0 &  1 &  0 &  0 &  2 \\
     0 & -1 &  1 &  0 &  2 \\
     0 & -1 & -1 &  1 &  2 \\
     0 & -1 & -1 & -1 &  2 
  \end{bmatrix}
$
&rarr; (*pivoting column 2 and 5*)
$
  \begin{bmatrix}
     1 &  1 &  0 &  0 &  0 \\
     0 &  2 &  0 &  0 &  1 \\
     0 &  2 &  1 &  0 & -1 \\
     0 &  2 & -1 &  1 & -1 \\
     0 &  2 & -1 & -1 & -1 
  \end{bmatrix}
$
&rarr;
$
  \begin{bmatrix}
     1 &  1 &  0 &  0 &  0 \\
     0 &  2 &  0 &  0 &  1 \\
     0 &  0 &  1 &  0 & -1 \\
     0 &  0 & -1 &  1 & -1 \\
     0 &  0 & -1 & -1 & -1 
  \end{bmatrix}
$
&rarr;
$
  \begin{bmatrix}
     1 &  1 &  0 &  0 &  0 \\
     0 &  2 &  0 &  0 &  1 \\
     0 &  0 &  1 &  0 & -1 \\
     0 &  0 &  0 &  1 &  0 \\
     0 &  0 &  0 & -1 &  0 
  \end{bmatrix}
$
&rarr;
$
  \begin{bmatrix}
     1 &  1 &  0 &  0 &  0 \\
     0 &  2 &  0 &  0 &  1 \\
     0 &  0 &  1 &  0 & -1 \\
     0 &  0 &  0 &  1 &  0 \\
     0 &  0 &  0 &  0 &  0 
  \end{bmatrix}
$
&rarr;
$
  \begin{bmatrix}
     1 &  1 &  0 &  0 &  0 \\
     0 &  1 &  0 &  0 &  \frac{1}{2} \\
     0 &  0 &  1 &  0 & -1 \\
     0 &  0 &  0 &  1 &  0 \\
     0 &  0 &  0 &  0 &  0 
  \end{bmatrix}
$
$= U$ 
\
\
Summing all transformations, we can retrieve the lower triangular matrix.
\
$
L =
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  0 \\
    -1 &  \frac{1}{2} &  0 &  0 &  0 \\
    -1 &  1 &  1 &  0 &  0 \\
    -1 &  1 & -1 &  1 &  0 \\
    -1 &  1 & -1 & -1 &  1 
  \end{bmatrix}    
$
\
\
When using complete pivoting, we did end up rearranging columns. Here, we were required to swap column 2 and 5 for the highest value to be in the pivot position. Hence, we have a permutation matrix $P$,
\
$
P =
  \begin{bmatrix}
     1 &  0 &  0 &  0 &  0 \\
     0 &  0 &  0 &  0 &  1 \\
     0 &  0 &  1 &  0 &  0 \\
     0 &  0 &  0 &  1 &  0 \\
     0 &  1 &  0 &  0 &  0 
  \end{bmatrix}    
$
\
Due to this, the entries in the matrix did not grow as we have seen in the partial pivoting. However, note that te last row of transformed matrix $A$ contains only zeros (including the pivot).
\
**TODO: Is it allowed to have a zero pivot for complete pivoting?**

## (b) (2 points)
Write a method that generates a matrix of the form of part (a) of size $n \times n$ for any $n$. Use a library routine for Gaussian elimination with partial pivoting to solve various sizes of linear systems of this form, using right-hand-side vectors chosen so that the solution is known. Try for example the case where the true solution is a vector of uniformly distributed random numbers between 0 and 1. How do the error, residual, and condition number behave as the systems become larger? Comment on the stability (see chapter 1) of Gaussian elimination with partial pivoting in this case.

N.B. This is an artificially contrived system that does not reflect the behavior of Gaussian elimination in realistic examples.

Logically, it is more likely for an error to occur in a relatively larger system since there are more values to compute and therefore a higher chance of a larger error. Hence, as the system gets larger (by increasing $n$), the error also increases in value.
\
**TODO residuals** *empty?*
\
**TODO condition number** *same as rank(A)? how?*
\
Here, the condition number is set to be calculated as described in lecture 1, i.e., taking the maximum value of the absolute values of the relative forward error over the relative backward error, $cond = max |\frac{error_{f_{rel}}}{error_{b_{rel}}}|$.
\
An algorithm is stable when it is insensitive to perturbations during computation. This is true when the condition number is small meaning that the problem is insensitive.

In [9]:
def generate_matrix(n):
    matrix = -1 * linalg.tril(np.ones((n, n)), k = -1)
    matrix += np.identity(n)
    matrix[:, -1] = 1
    return matrix

In [75]:
def calc_cond_error(matrix, solution):

    print("condition number: ", np.linalg.cond(matrix, p=np.inf) )

    b = matrix @ solution      # with roundoff error order 1e-16

    # LU decomposition with partial pivoting
    lu, piv = linalg.lu_factor(matrix)
    estimate = linalg.lu_solve((lu, piv), b)
    print("error: ", np.linalg.norm(estimate-solution)) 
    lstsq = np.linalg.lstsq(matrix,solution)
    print("least squares solution: ", lstsq[0])
    print("residuals: ", lstsq[1]) # residuals, empty if rank > matrix.shape[0] or < matrix.shape[1]
    print("rank: ", lstsq[2])
    print(lstsq[2] == matrix.shape[0])
    print("singular values of A: ", lstsq[3])

In [76]:
n = 10
A = generate_matrix(n)
solution = np.random.uniform(0,1,n)
calc_cond_error(A,solution)

condition number:  10.0
error:  7.874761121542543e-15
least squares solution:  [-0.04576427  0.05522106  0.12654036 -0.05653685 -0.39977211  0.05443163
 -0.06939655  0.111167   -0.1338834   0.50973988]
residuals:  []
rank:  10
True
singular values of A:  [6.20128722 2.44582866 1.8512543  1.65764412 1.57522551 1.53470767
 1.51356927 1.50315584 1.41421356 1.41421356]


/var/folders/zb/qwtwsp1d539dg2nj7t84r7mc0000gn/T/ipykernel_35739/1141070959.py:11: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  lstsq = np.linalg.lstsq(matrix,solution)


In [59]:
n = 20
A = generate_matrix(n)
solution = np.random.uniform(0,1,n)
calc_cond_error(A,solution)

condition number:  20.0
error:  8.651048809155353e-12
solution 'x':  [-0.04559847  0.28164329  0.04479254 -0.20696325  0.12723513 -0.08435655
 -0.09012901  0.07985554 -0.09824926 -0.11048393  0.2044255  -0.16802361
  0.11849287 -0.18510731  0.22688596 -0.37161977 -0.00067646  0.08261648
 -0.19157648  0.57108083]
residuals:  []
rank:  20
singular values of A:  [12.49364201  4.37159227  2.85691052  2.27030923  1.98074933  1.8179875
  1.718398    1.65362485  1.60951466  1.57840437  1.55587806  1.53926224
  1.52687343  1.51761888  1.51077424  1.505855    1.50254008  1.50062599
  1.41421356  1.41421356]


/var/folders/zb/qwtwsp1d539dg2nj7t84r7mc0000gn/T/ipykernel_35739/3639385442.py:11: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  lstsq = np.linalg.lstsq(matrix,solution)


In [56]:
n = 50
A = generate_matrix(n)
solution = np.random.uniform(0,1,n)
calc_cond_error(A,solution)

condition number:  50.0
error:  0.004177424359615524
solution 'x':  [ 8.26580277e-02 -5.16093283e-02 -7.23502488e-02  2.24153445e-01
 -1.33642630e-01 -4.12101987e-01  4.19489491e-02  1.54344580e-01
  1.39571133e-01  3.26299045e-01 -1.17663167e-01 -1.24713008e-01
 -3.49207729e-01  1.07889172e-01  2.15243512e-01  5.47693894e-02
 -1.99870094e-01  2.91464182e-01 -2.06397537e-01  1.46555676e-01
 -2.03738678e-01 -1.33188494e-01  2.07089695e-01  2.40614200e-01
 -1.29741595e-01  5.85519320e-06 -6.30678341e-02 -2.54224854e-02
  1.75959241e-01 -1.53225927e-01  7.81535987e-02 -1.61064200e-01
  1.03539357e-01  3.61796183e-02 -3.68970082e-01 -3.60481059e-02
 -9.46159301e-02  5.65214381e-02  8.96281183e-02  1.95510286e-01
 -2.04450747e-01  1.16733178e-01 -1.12575980e-01 -1.38987735e-02
 -1.66399002e-02  1.79827493e-01  2.50232712e-01 -3.39658696e-02
  4.47974529e-02  5.20125001e-01]
residuals:  []
rank:  50
singular values of A:  [31.54507733 10.60104603  6.46267662  4.72352762  3.78242508  3.202434

/var/folders/zb/qwtwsp1d539dg2nj7t84r7mc0000gn/T/ipykernel_35739/3639385442.py:11: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  lstsq = np.linalg.lstsq(matrix,solution)


In [57]:
n = 100
A = generate_matrix(n)
solution = np.random.uniform(0,1,n)
calc_cond_error(A,solution)

condition number:  100.0
error:  3.4507630019224056
solution 'x':  [-0.30321304  0.26835443  0.0900944  -0.00866842 -0.02810324  0.04474865
 -0.06615892 -0.29113714  0.23996848 -0.12167899 -0.21345072  0.18160642
 -0.0848879   0.01836378  0.1231829   0.04518863 -0.07410807  0.12804231
  0.05160368  0.10588982 -0.13560881 -0.15419138  0.31195404 -0.26534192
 -0.00690696 -0.06593244  0.0405439   0.20098951 -0.1232475  -0.10713301
  0.36114516 -0.12745352 -0.03883313  0.09801246  0.03940384 -0.07016612
 -0.16396693 -0.38876063  0.08232757 -0.01435601  0.11469379  0.16049529
  0.16998629  0.17635947 -0.18747792 -0.28817672  0.25927963 -0.04562265
  0.28823278 -0.04634117 -0.21519316  0.30261487 -0.14068433  0.02798721
 -0.16984716 -0.16940478  0.11425755 -0.08688246  0.09721204  0.20069951
 -0.07018785 -0.23000998  0.17440794 -0.12010455 -0.06020769  0.01302108
  0.08160673  0.16816241 -0.02362168 -0.13697275  0.12750583 -0.29509951
  0.27092831  0.05166559  0.03976274 -0.22404012  0.10130

/var/folders/zb/qwtwsp1d539dg2nj7t84r7mc0000gn/T/ipykernel_35739/3639385442.py:11: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  lstsq = np.linalg.lstsq(matrix,solution)
